In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/down_sample.csv')[['Subreddit', 'simp_lem']]

In [ ]:
df.head()

,Subreddit,simp_lem
0,0,i hate how i feel when i get interested in som...
1,0,how to make friend can someone suggest somethi...
2,0,so today wa my birthday 9 11 and a i wa hoping...
3,0,having a hard time just need someone to say i ...
4,0,anti depressant inducing trip like state any o...


In [ ]:
X = df.drop(columns='Subreddit')
y = df['Subreddit']

X_train, X_test, y_train, y_test=train_test_split(X, y, stratify = y, random_state=42)

In [ ]:
cvec = CountVectorizer()
cvec.fit(X_train['simp_lem'])
X_train_cv = pd.DataFrame(cvec.transform(X_train['simp_lem']).toarray(), columns = cvec.get_feature_names())
X_test_cv = pd.DataFrame(cvec.transform(X_test['simp_lem']).toarray(), columns = cvec.get_feature_names())

tvec = TfidfVectorizer()
tvec.fit(X_train['simp_lem'])
X_train_tv = pd.DataFrame(tvec.transform(X_train['simp_lem']).todense(), columns = tvec.get_feature_names())
X_test_tv = pd.DataFrame(tvec.transform(X_test['simp_lem']).todense(), columns = tvec.get_feature_names())

In [ ]:
sc = StandardScaler()
sc.fit(X_train_tv)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
X_train_tv_sc = sc.transform(X_train_tv)

In [ ]:
X_test_tv_sc = sc.transform(X_test_tv)

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X_train_tv_sc.shape[1], kernel_regularizer=l2(.001)))
model.add(Dropout(0.50))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()], optimizer='adam')

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, 
                           verbose=1, mode='auto')

In [ ]:
history = model.fit(X_train_tv_sc, y_train, 
                    validation_data=(X_test_tv_sc, y_test),
                    epochs=50, 
                    batch_size=64,
                    verbose=1, 
                    callbacks=[early_stop])

Epoch 1/50
352/352 [==============================] - 4s 10ms/step - loss: 1.5093 - accuracy: 0.5311 - precision_1: 0.5282 - val_loss: 0.9120 - val_accuracy: 0.6199 - val_precision_1: 0.5953
Epoch 2/50
352/352 [==============================] - 2s 7ms/step - loss: 0.9966 - accuracy: 0.5976 - precision_1: 0.5872 - val_loss: 0.8936 - val_accuracy: 0.7007 - val_precision_1: 0.7143
Epoch 3/50
352/352 [==============================] - 2s 7ms/step - loss: 0.9040 - accuracy: 0.6800 - precision_1: 0.6975 - val_loss: 0.8561 - val_accuracy: 0.7231 - val_precision_1: 0.7756
Epoch 4/50
352/352 [==============================] - 2s 7ms/step - loss: 0.8274 - accuracy: 0.7458 - precision_1: 0.7935 - val_loss: 0.8216 - val_accuracy: 0.7253 - val_precision_1: 0.7866
Epoch 5/50
352/352 [==============================] - 2s 7ms/step - loss: 0.7343 - accuracy: 0.7947 - precision_1: 0.8409 - val_loss: 0.8267 - val_accuracy: 0.7289 - val_precision_1: 0.7667
Epoch 6/50
352/352 [=============================